In [1]:
## learn fastai's mid level API for processing data

In [2]:
%%time
import fastai
from fastai.text.all import *
from pathlib import Path

CPU times: user 2.08 s, sys: 1.2 s, total: 3.28 s
Wall time: 3.22 s


In [3]:
path = untar_data(URLs.IMDB)

In [4]:
path

Path('/home/.fastai/data/imdb')

In [5]:
%%time
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

CPU times: user 696 ms, sys: 31.6 ms, total: 728 ms
Wall time: 726 ms


In [6]:
txts = L(o.open().read() for o in files[:2000])

In [7]:
txts[0]

"I didn't know what to expect when I started watching this movie, by the end of it I was pulling my hairs out. This was one of the most pathetic movies of this year...in fact, in the last ten years. David Dhawan should just give up his career as a director. I am yet to come across one original script that David Dhawan has worked on. This one was a complete bit y bit rip off Hitch. I have nothing against remakes as such, but this one is just so lousy that it makes you even hate the original one (which was pretty decent). I fail to understand what actors like Salman and Govinda saw in this script. I read somewhere, that this was supposed to be Govinda's comeback vehicle. If thats true, then only God can save his career. Salman just overacted to the hilt. Govinda who I think is an actor of very high caliber was completely wasted. Katrina Kaif and LAra Dutta had nothing to do apart form wearing designer clothes and smiling for no rhyme or reason. Please stay away form this one!"

In [8]:
tok = Tokenizer.from_folder(path)
tok.setup(txts)
toks = txts.map(tok)
toks[0]

(#231) ['xxbos','i','did',"n't",'know','what','to','expect','when','i'...]

In [9]:
num = Numericalize()
num.setup(toks)
nums = toks.map(num)
nums[0][:10]

TensorText([  2,  17,  80,  32, 131,  63,  15, 530,  75,  17])

In [10]:
nums_dec = num.decode(nums[0][:10]); nums_dec

(#10) ['xxbos','i','did',"n't",'know','what','to','expect','when','i']

In [11]:
tok.decode(nums_dec)

"xxbos i did n't know what to expect when i"

In [12]:
## Both Tokenize and Numericalize are Transforms

In [13]:
## Transforms is always applied on TUPLES (input, target)

In [14]:
tok((txts[0], txts[1]))

((#231) ['xxbos','i','did',"n't",'know','what','to','expect','when','i'...],
 (#260) ['xxbos','xxmaj','for','several','reasons',',','this','movie','is','simply'...])

In [15]:
## Writing your own Transform

In [16]:
## x:int means transform will only be applied when it is an integer
def f(x:int): return x + 1

In [17]:
tfm = Transform(f)
tfm(2), tfm(2.0)

(3, 2.0)

In [18]:
def f(x:str) : return "hello" + x

In [19]:
tfm = Transform(f)
tfm("sadiva"), tfm(2)

('hellosadiva', 2)

In [20]:
@Transform
def f(x:int): return x + 3

In [21]:
f(4), f(4.00)

(7, 4.0)

In [22]:
## @ - special syntax in python to pass any function(f) to another function(Transform)

In [23]:
## If we want setup or decode function, then we need to subclass Transform

In [24]:
class NormalizeMean(Transform):
    def setups(self, items):
        self.mean = sum(items)/len(items)
    def encodes(self, x):
        return x - self.mean
    def decodes(self, x):
        return x + self.mean

In [25]:
tfm = NormalizeMean()

In [26]:
tfm.setup([1,2,3,4])

In [27]:
start = 2
y = tfm(start)

In [28]:
tfm.mean

2.5

In [29]:
y

-0.5

In [30]:
start = 3
y = tfm(start)

In [31]:
y

0.5

In [32]:
start = 4
y = tfm(start)
y

1.5

In [33]:
z = tfm.decode(start)

In [34]:
z

6.5

In [35]:
## Transform () - encodes
## Transform decode() - decodes
## Transform setup() - setups

In [36]:
## Pipeline - used to compose several transforms together 

In [37]:
tfms = Pipeline([tok, num])
t = tfms(txts[0]); t[:20]

TensorText([  2,  17,  80,  32, 131,  63,  15, 530,  75,  17, 521, 158,  20,  27,
         11,  58,   9, 143,  14,  18])

In [38]:
tfms.decode(t)[:100]

"xxbos i did n't know what to expect when i started watching this movie , by the end of it i was pull"

In [39]:
## The only part that doesn't work the same way as Transform in Pipeline is setup. For that we need TfmdLists

In [40]:
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize()])

In [41]:
t = tls[0] ; t[:20]

TensorText([  2,  19,  89,  36, 141,  65,  15, 534,  69,  19, 683, 170,  20,  30,
         11,  48,   9, 149,  14,  17])

In [42]:
## TfmdLists will call the setup method of each Transform in order

In [43]:
tls.decode(t)[:100]

"xxbos i did n't know what to expect when i started watching this movie , by the end of it i was pull"

In [44]:
tls.show(t)

xxbos i did n't know what to expect when i started watching this movie , by the end of it i was pulling my hairs out . xxmaj this was one of the most pathetic movies of this year … in fact , in the last ten years . xxmaj david xxmaj dhawan should just give up his career as a director . i am yet to come across one original script that xxmaj david xxmaj dhawan has worked on . xxmaj this one was a complete bit y bit rip off xxmaj hitch . i have nothing against remakes as such , but this one is just so lousy that it makes you even hate the original one ( which was pretty decent ) . i fail to understand what actors like xxmaj salman and xxmaj govinda saw in this script . i read somewhere , that this was supposed to be xxmaj govinda 's comeback vehicle . xxmaj if that s true , then only xxmaj god can save his career . xxmaj salman just overacted to the hilt . xxmaj govinda who i think is an actor of very high caliber was completely wasted . xxmaj katrina xxmaj kaif and lara xxmaj dutta had n

In [45]:
##  At initialization TfmdLists will automatically call the setup method of each transform in order

In [46]:
## TfmdLists can also handle training and validation splits

In [49]:
cut = int(len(files)*0.8)
splits = [list(range(cut)), list(range(cut, len(files)))]
len(splits[0]), len(splits[1])

(80000, 20000)

In [50]:
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize],
                splits = splits)

In [51]:
tls.valid[0][:20]

TensorText([    2,     8,    20,    16,    38,    13,    32,    33,    80,    85,
        22127, 12250,    51,    17,    23,   391,    10,     8,     9,   331])

In [52]:
tls.train[1][:20]

TensorText([   2,    8,   29,  457, 1002,   11,   20,   30,   16,  352,  399,   10,
           8,  102, 5803,   42, 3872,   66,   14,   20])

In [53]:
lbls = files.map(parent_label)
lbls

(#100000) ['neg','neg','neg','neg','neg','neg','neg','neg','neg','neg'...]

In [54]:
cat = Categorize()
cat.setup(lbls)
cat.vocab

['neg', 'pos', 'unsup']

In [55]:
cat(lbls[-1])

TensorCategory(1)

In [56]:
cat(lbls[0])

TensorCategory(0)

In [57]:
tls_y = TfmdLists(files, [parent_label, Categorize()])
tls_y[0]

TensorCategory(0)

In [58]:
tls_y[-1]

TensorCategory(1)

In [59]:
## Datasets

In [60]:
## Datasets will apply two or more pipelines in parallel to the same raw object

In [64]:
x_tfms = [Tokenizer.from_folder(path), Numericalize]
y_tfms = [parent_label, Categorize()]
dsets = Datasets(files, [x_tfms, y_tfms], splits = splits)
x,y = dsets[0]

In [65]:
x[:20], y

(TensorText([  2,  19,  89,  36, 141,  65,  15, 534,  69,  19, 683, 170,  20,  30,
          11,  48,   9, 149,  14,  17]),
 TensorCategory(0))

In [66]:
len(dsets.train), len(dsets.valid)

(80000, 20000)

In [67]:
t = dsets.valid[0]
dsets.decode(t)

("xxbos xxmaj this is not a film you can really analyse separately from it 's production . xxmaj the audience became the film - makers to an extent unprecedented in the history of the xxmaj american film industry ; we felt so involved that viewing it becomes like watching the work of a friend . xxmaj how is it possible to be objective ? xxmaj this is our movie , is n't it ? xxmaj or is it ? xxmaj there may be nothing more disingenuous than a film - maker who promotes himself as the audience 's friend , giving them all the naughty treats that the xxunk critics would deny them . xxmaj just look at that prime self - publicist xxmaj eli xxmaj roth , promising gore - hounds all the viscera missing from literally gutless mainstream horror films , only to churn out a watered down and technically incompetent piece of work like ' hostel ' . \n\n xxmaj david xxup r. xxmaj ellis may not have spawned the monster that was the internet response to his film , but he was , quite understandably , quick

In [68]:
## last step is to convert datasets to dataloaders

In [69]:
dls = dsets.dataloaders(bs =64, before_batch = pad_input)

In [70]:
help(pad_input)

Help on Pad_Input in module fastai.text.data object:

class Pad_Input(fastcore.transform.ItemTransform)
 |  Pad_Input(self, enc=None, dec=None, split_idx=None, order=None)
 |  
 |  A transform that always take tuples as items
 |  
 |  Method resolution order:
 |      Pad_Input
 |      fastcore.transform.ItemTransform
 |      fastcore.transform.Transform
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  decodes = (TensorText,object) -> decodes
 |  
 |  encodes = (object,object) -> encodes
 |  
 |  setups = 
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __signature__ = <Signature (self, enc=None, dec=None, split_idx=None, ...
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from fastcore.transform.ItemTransform:
 |  
 |  __call__(self, x, **kwargs)
 |      Call self as a function.
 |  
 |  decode(self, x, **kwargs)
 |      Delegate to <code>

In [72]:
dls.train

In [73]:
dls.show_batch()

,text,category
0,"xxbos xxmaj this movie was great . i do n't know why all of these people thought it was n't good . xxmaj if you have n't seen this movie , i advise that you should watch it . xxmaj you will certainly laugh and you 'll certainly have a lot of fun . xxmaj steve xxmaj martin is great , and so are xxmaj goldie xxmaj hawn and xxmaj john xxmaj cleese . \n\n i rated it a perfect 10 ! xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad",unsup
1,"xxbos i read the comments on here and went in expecting something really good . \n\n i was so very disappointed by this movie . \n\n i really like this movie on one level but i think it could be a xxup lot better . \n\n xxmaj something was lacking . xxmaj the characters do n't seem to click , the ending felt incomplete and there was not enough relationship development . xxmaj why did they have a topless scene is beyond my comprehension . \n\n xxmaj the acting was good , there was enough humour to keep me smiling and the music was heart warming . \n\n i totally recommend the movie to everyone but i only give it an e for xxmaj effort . \n\n xxmaj if you are reading my comment , ignore how others are over - hyping this movie . xxmaj expect only a 6",unsup
2,"xxbos i know a lot of you hate these xxunk prequels ) , but seriously , this one was great ! xxmaj it explains what happens between after xxmaj bambi 's mother 's death and before he grows up , and focuses on xxmaj bambi and his father , xxmaj the xxmaj great xxmaj prince xxmaj of xxmaj the xxmaj forest . \n\n i found it entertaining , not wandering off the original 's storyline too much , and overall really good . xxmaj it 's better than you expect it to be . \n\n i do n't know why people hate sequels and prequels so much . xxmaj to be honest , i think most people do n't give them a chance . xxmaj but you should give this one a chance , it 's not disappointing . \n\n xxmaj i 'm really fond of the songs . xxmaj",unsup
3,"xxbos xxmaj in xxmaj world xxmaj war xxup ii , during a storm , a small plane crashes in an island between xxmaj cuba and xxmaj porto xxmaj rico , but the pilot xxmaj james ' mac ' mccarthy ( dick xxmaj purcell ) and his two passengers , xxmaj bill xxmaj summers ( john xxmaj archer ) and his black servant xxmaj jeff xxmaj jackson ( mantan xxmaj moreland ) , survive . xxmaj bill is in a mission trying to find xxmaj admiral xxmaj arthur xxmaj wainwright ( guy xxmaj usher ) , who vanished in that area . xxmaj they are lodged by the xxmaj austrian xxmaj dr . xxmaj mikhail xxmaj sangre ( henry xxmaj victor ) in his mansion . xxmaj the scared xxmaj jeff finds that the place has zombies , but xxmaj mac and xxmaj bill do not believe on him . xxmaj",unsup
4,"xxbos i ca n't believe that someone actually paid to have this film made . xxmaj stupid , unrealistic , and stereotypical . xxmaj right from the take off of the massive 747 the pilot pulled the throttles back to increase speed . then you have 5 armed persons with semi to fully automatic weapons firing without so much as one bullet breaching the walls of the pressurized cabin at 38 , xxrep 3 0 feet . xxmaj then once below in the belly of the plane a stray bullet hits a xxup fuel line and we see the fuel leaking from the side of the plane . xxmaj the acting was just horrid and forced . xxmaj there just did n't seem to be any direction . i have seen some pretty horrid b movies in my lifetime but with the names that were in this film i was",neg
5,"xxbos xxmaj as far as movies go , this one has what is popular : action , adventure , sex , blood , and lots of twists that will keep you guessing what the truth is . \n\n x

In [74]:
## putting it all together

In [75]:
tfms = [[Tokenizer.from_folder(path), Numericalize], [parent_label, Categorize()]]
files = get_text_files(path, folders = ['train', 'test'])
splits = GrandparentSplitter(valid_name = 'test')(files)
dsets = Datasets(files, tfms, splits = splits)
dls = dsets.dataloaders(dl_type = SortedDL, before_batch = pad_input)

In [76]:
files

(#50000) [Path('/home/.fastai/data/imdb/test/neg/6201_1.txt'),Path('/home/.fastai/data/imdb/test/neg/11881_2.txt'),Path('/home/.fastai/data/imdb/test/neg/11690_1.txt'),Path('/home/.fastai/data/imdb/test/neg/7300_1.txt'),Path('/home/.fastai/data/imdb/test/neg/2931_1.txt'),Path('/home/.fastai/data/imdb/test/neg/11165_4.txt'),Path('/home/.fastai/data/imdb/test/neg/1026_2.txt'),Path('/home/.fastai/data/imdb/test/neg/11499_3.txt'),Path('/home/.fastai/data/imdb/test/neg/3201_1.txt'),Path('/home/.fastai/data/imdb/test/neg/429_3.txt')...]

In [78]:
## dl_type = SortedDL means that it constructs batches by putting samples of roughly the same size in similar batches

In [79]:
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxmaj heavy - handed moralism . xxmaj writers using characters as mouthpieces to speak for themselves . xxmaj predictable , plodding plot points ( say that five times fast ) . a child 's imitation of xxmaj britney xxmaj spears . xxmaj this film has all the earmarks of a xxmaj lifetime xxmaj special reject . \n\n i honestly believe that xxmaj jesus xxmaj xxunk and xxmaj julia xxmaj xxunk set out to create a thought - provoking , emotional film on a tough subject , exploring the idea that things are not always black and white , that one who is a criminal by definition is not necessarily a bad human being , and that there can be extenuating circumstances , especially when one puts the well - being of a child first . xxmaj however , their earnestness ends up being channeled into preachy dialogue and trite",neg
3,"xxbos xxmaj okay , so xxmaj i 'm not a big video game buff , but was the game xxmaj house of the xxmaj dead really famous enough to make a movie from ? xxmaj sure , they went as far as to actually put in quick video game clips throughout the movie , as though justifying any particular scene of violence , but there are dozens and dozens of games that look exactly the same , with the hand in the bottom on the screen , supposedly your own , holding whatever weapon and goo - ing all kinds of aliens or walking dead or snipers or whatever the case may be . \n\n xxmaj it 's an interesting premise in xxmaj house of the xxmaj dead , with a lot of college kids ( loaded college kids , as it were , kids who are able to pay",neg
4,"xxbos xxmaj chris xxmaj rock deserves better than he gives himself in "" down xxmaj to xxmaj earth . "" xxmaj as directed by brothers xxmaj chris & xxmaj paul xxmaj weitz of "" american xxmaj pie "" fame , this uninspired remake of xxmaj warren xxmaj beatty 's 1978 fantasy "" heaven xxmaj can xxmaj wait , "" itself a rehash of 1941 's "" here xxmaj comes xxmaj mr . xxmaj jordan , "" lacks the xxunk profane humor that won xxmaj chris xxmaj rock an xxmaj emmy for his first xxup hbo special . xxmaj predictably , he spouts swear words from a to xxup z , but he consciously avoids the xxmaj f - word . xxmaj anybody who saw this gifted african - american comic in "" lethal xxmaj weapon 4 , "" "" dogma , "" or "" nurse xxmaj betty "" knows he",neg
5,"xxbos i have never seen any of xxmaj spike xxmaj lee